In [1]:
from pyDeepInsight import ImageTransformer, CAMFeatureSelector

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import torch
import torchvision.transforms as transforms
import timm
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from geomstats.learning.frechet_mean import FrechetMean
from geomstats.learning.pca import TangentPCA
from geomstats.geometry.hypersphere import Hypersphere
from IPython.display import display, clear_output
import torchvision.models as models
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import roc_auc_score, ConfusionMatrixDisplay, confusion_matrix
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, roc_auc_score, roc_curve
import warnings

warnings.filterwarnings("ignore")

INFO: Using numpy backend


In [2]:
data = pd.read_csv('ibd_data_set.csv',index_col=0)
data = data.div(data.sum(axis=1), axis=0) #compositional data로 조정
data = np.sqrt(data) #square-root 변환
X=data.values #X data로 지정
sample=pd.read_csv('ibd_response.csv',index_col=0)
y=sample['ibd'] #label 지정
le = LabelEncoder()
y_enc = le.fit_transform(y)
num_classes = np.unique(y_enc).size

In [3]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score
from sklearn.utils import shuffle
import optuna

# 5-Fold Cross-Validation 적용
kf = KFold(n_splits=10, shuffle=True)

# 정확도 및 AUC 저장 리스트
accuracies = []
aucs = []

le = LabelEncoder()  # 라벨 인코더

pixel_size = (224, 224)
sphere = Hypersphere(dim=36348)  # Tangent PCA 적용을 위해 실제 차원보다 한 차원 낮은 Hypersphere 선언
it = ImageTransformer(
    feature_extractor=TangentPCA(sphere, n_components=2),
    pixels=pixel_size)  # TangentPCA의 기본 중심점은 frechet mean

# Optuna Study 생성
study = optuna.create_study(direction='maximize')

# Optuna에서 lr과 weight_decay를 최적화하기 위해 트라이얼 생성
def optimize_trial(trial):
    # 학습률과 weight_decay를 Optuna에서 최적화할 하이퍼파라미터로 설정
    lr = trial.suggest_float('lr', 1e-6, 1e-2, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-2, log=True)
    
    actual_labels_list = []
    probs_list = []
    
    for train_index, test_index in kf.split(X):
        # Train/Test Split
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # ImageTransformer 적용 (예외 처리 추가)
        try:
            it.fit(X_train, y=y_train, plot=False)
        except Exception as e:
            print(f"SVD 수렴 문제 발생: {e}. 계속 진행합니다.")
            continue  # 수렴 문제 발생 시 다음 루프로 넘어감

        X_train_img = it.transform(X_train)
        X_test_img = it.transform(X_test)

        fdm = it.feature_density_matrix()
        fdm_nonzero_indices = np.nonzero(fdm)
        X_train_img = it.transform(X_train)
        X_test_img = it.transform(X_test)
        epsilon = 1e-8
        for i in range(len(X_train_img)):
            for channel in range(3):
                X_train_img[i][fdm_nonzero_indices[0], fdm_nonzero_indices[1], channel] += epsilon

        preprocess = transforms.Compose([
            transforms.ToTensor()
        ])

        # Tensor로 변환
        X_train_tensor = torch.stack([preprocess(img) for img in X_train_img]).float()
        y_train_tensor = torch.from_numpy(le.fit_transform(y_train))

        X_test_tensor = torch.stack([preprocess(img) for img in X_test_img]).float()
        y_test_tensor = torch.from_numpy(le.transform(y_test))

        # DataLoader 설정
        batch_size = 32
        trainset = TensorDataset(X_train_tensor, y_train_tensor)
        trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)

        testset = TensorDataset(X_test_tensor, y_test_tensor)
        testloader = DataLoader(testset, batch_size=1, shuffle=False)

        # EfficientNet B0 모델을 로드하고 이진 분류를 위한 구조로 수정
        #model = models.efficientnet_b0(weights='DEFAULT')
        # 마지막 fully connected layer를 이진 분류용으로 수정
        #num_ftrs = model.classifier[1].in_features
        #model.classifier[1] = nn.Linear(num_ftrs, 1)  # 이진 분류용 출력층
        model = models.resnet50(weights='DEFAULT')
        
        # 마지막 fully connected layer를 이진 분류용으로 수정
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, 1)  # 이진 분류용 출력층
        
        # 손실 함수 및 옵티마이저 설정
        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

        # 모델을 GPU로 이동
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)

        # 학습 시작
        epochs = 100  # 최적화 탐색을 위해 에포크 수를 적게 설정
        for epoch in range(epochs):
            model.train()
            running_loss = 0.0

            for i, data in enumerate(trainloader, 0):
                inputs, labels = data
                inputs = inputs.to(device)
                labels = labels.to(device).float()

                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels.unsqueeze(1))
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
        
        # 평가 모드로 전환
        model.eval()

        with torch.no_grad():
            for data in testloader:
                inputs, labels = data
                inputs = inputs.to(device)
                labels = labels.to(device).float()

                outputs = model(inputs)
                probs = torch.sigmoid(outputs).cpu().numpy()

                # 실제 레이블 및 확률 값 저장 (AUC 계산용)
                actual_labels_list.append(labels.cpu().numpy()[0])
                probs_list.append(probs[0])

    # AUC 계산
    try:
        auc = roc_auc_score(actual_labels_list, probs_list)
    except ValueError:
        # AUC 계산이 실패하는 경우 예외 처리
        print("AUC 계산에 실패했습니다.")
        auc = 0.0

    # Optuna는 목적 함수를 최대화해야 하므로 AUC 반환
    return auc

# Optuna 최적화 실행
study.optimize(optimize_trial, n_trials=1000)

# 최적화 결과 출력
print(f'Best trial AUC: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

[I 2024-09-30 12:46:36,870] A new study created in memory with name: no-name-4eeabe1a-b4e7-49a0-9a85-99a87f5cd595


SVD 수렴 문제 발생: SVD did not converge. 계속 진행합니다.


[I 2024-09-30 13:20:07,094] Trial 0 finished with value: 0.5746527777777777 and parameters: {'lr': 8.026698630342224e-06, 'weight_decay': 9.877164367181819e-05}. Best is trial 0 with value: 0.5746527777777777.


SVD 수렴 문제 발생: SVD did not converge. 계속 진행합니다.


[I 2024-09-30 13:53:47,634] Trial 1 finished with value: 0.5137096774193548 and parameters: {'lr': 0.0001063337754565088, 'weight_decay': 1.0683771457349063e-05}. Best is trial 0 with value: 0.5746527777777777.


...

SVD 수렴 문제 발생: SVD did not converge. 계속 진행합니다.


[I 2024-10-05 19:44:46,723] Trial 998 finished with value: 0.6983870967741935 and parameters: {'lr': 2.276271849974255e-05, 'weight_decay': 0.0005036870292967698}. Best is trial 483 with value: 0.8469945355191256.


SVD 수렴 문제 발생: SVD did not converge. 계속 진행합니다.


[I 2024-10-05 19:54:32,954] Trial 999 finished with value: 0.5280303030303031 and parameters: {'lr': 0.0008374075362027547, 'weight_decay': 0.0008938366362920961}. Best is trial 483 with value: 0.8469945355191256.


Best trial AUC: 0.8469945355191256
Best hyperparameters: {'lr': 2.7954302132304642e-05, 'weight_decay': 0.000786097250513217}
